In [1]:
import tensorflow as tf
import mnist_inference
import Layer2Model
import os
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
BATCH_SIZE = 100 
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30001
MOVING_AVERAGE_DECAY = 0.99 
MODEL_SAVE_PATH = "../model/{}".format(pd.Timestamp.now())[:19]
MODEL_NAME = "tianchi_model"
two_layers_model_fileName = '2HiddenLayer'
one_layers_model_fileName = '1HiddenLayer'
TENSORBOARD_LOG = 'tensor_board'
TrainDataFileName = 'withMax_trainData.csv'

In [3]:
def startTrain(trainX, trainY, model_path, model_name, model):
    dataSize = len(trainY)
    
    with tf.device('/device:GPU:0'):
    # 定义输入输出placeholder。
        x = tf.placeholder(tf.float32, [None, model.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, model.OUTPUT_NODE], name='y-input')

        regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
        y = model.inference(x, regularizer)
        global_step = tf.Variable(0, trainable=False)
    
    # 定义损失函数、学习率、滑动平均操作以及训练过程。
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
#     cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
#     cross_entropy_mean = tf.reduce_mean(cross_entropy)
        beginLoss = tf.sqrt(tf.reduce_mean(tf.pow(tf.subtract(y, y_), 2)))
#     loss = beginLoss + tf.add_n(tf.get_collection('losses'))
        loss = beginLoss
        learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            dataSize / BATCH_SIZE, LEARNING_RATE_DECAY,
            staircase=True)
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        with tf.control_dependencies([train_step, variables_averages_op]):
            train_op = tf.no_op(name='train')
        
    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    config = tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
    with tf.Session(config = config) as sess:
        tf.global_variables_initializer().run()
        
        head = 0
        for i in range(TRAINING_STEPS):
            tail = head+BATCH_SIZE
            if tail > dataSize:
                xs = np.concatenate((trainX[head: BATCH_SIZE], trainX[0: tail-BATCH_SIZE]))
                ys = np.concatenate((trainY[head: BATCH_SIZE], trainY[0: tail-BATCH_SIZE]))
                head = tail - BATCH_SIZE
            else:
                xs, ys = trainX[head: head+BATCH_SIZE-1], trainY[head: head+BATCH_SIZE-1]
                head = tail
            
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i % 1000 == 0:
                print("After", step," training step(s), loss on training batch is ", loss_value)
            if i % 10000 == 0:
                saver.save(sess, os.path.join(model_path, model_name), global_step=global_step)
                testLoss = sess.run([loss], feed_dict={x: testX, y_: testY})
                print("After", step," training step(s), loss on test set is ", testLoss)

In [4]:
# handle data, produce train and test input

df = pd.read_csv(os.path.join('../CleanData/',TrainDataFileName))
# random shift the df
df = df.sample(frac=1).reset_index(drop=True)

normalizeColumns = ['compartment','TR','displacement','price_level','power','level_id',
                    'cylinder_number','engine_torque','car_length','car_height','car_width','total_quality','equipment_quality',
                    'rated_passenger','wheelbase','front_track','rear_track']
leftDf = df.drop(normalizeColumns, axis =1 ).drop(['sale_quantity'], axis = 1)

normalizeDf = df[normalizeColumns]
normalizeDf = (normalizeDf-normalizeDf.min())/(normalizeDf.max()-normalizeDf.min())
inputDf = pd.concat([leftDf, normalizeDf], axis = 1)
inputX = inputDf.values
resultArray = df['sale_quantity'].values
inputY = resultArray.reshape((len(resultArray),1))
trainX, testX, trainY, testY = train_test_split(inputX, inputY, test_size=0.1, random_state=42)

In [5]:
def continueTrain(trainX, trainY, sess, continue_steps):
    head = 0
    for i in range(continue_steps):
        tail = head+BATCH_SIZE
        if tail > dataSize:
            xs = np.concatenate((trainX[head: BATCH_SIZE], trainX[0: tail-BATCH_SIZE]))
            ys = np.concatenate((trainY[head: BATCH_SIZE], trainY[0: tail-BATCH_SIZE]))
            head = tail - BATCH_SIZE
        else:
            xs, ys = trainX[head: head+BATCH_SIZE-1], trainY[head: head+BATCH_SIZE-1]
            head = tail
            
        _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
        if i % 1000 == 0:
            print("After", step," training step(s), loss on training batch is ", loss_value)
        if i % 10000 == 0:
            saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)
            testLoss = sess.run([loss], feed_dict={x: testX, y_: testY})
            print("After", step," training step(s), loss on test set is ", restoredSession.run([loss], feed_dict={x: testX, y_: testY}))

In [6]:
new_path = os.path.join(MODEL_SAVE_PATH, one_layers_model_fileName)
if not os.path.exists(new_path):
    os.makedirs(new_path)
startTrain(trainX, trainY, MODEL_SAVE_PATH, one_layers_model_fileName, mnist_inference)

After 1  training step(s), loss on training batch is  419.96
After 1  training step(s), loss on test set is  [417.6713]
After 1001  training step(s), loss on training batch is  353.274
After 2001  training step(s), loss on training batch is  311.271
After 3001  training step(s), loss on training batch is  285.99
After 4001  training step(s), loss on training batch is  268.561
After 5001  training step(s), loss on training batch is  255.727
After 6001  training step(s), loss on training batch is  246.769
After 7001  training step(s), loss on training batch is  241.195
After 8001  training step(s), loss on training batch is  234.767
After 9001  training step(s), loss on training batch is  230.813
After 10001  training step(s), loss on training batch is  227.408
After 10001  training step(s), loss on test set is  [212.97559]
After 11001  training step(s), loss on training batch is  224.829
After 12001  training step(s), loss on training batch is  222.763
After 13001  training step(s), los

In [7]:
# # 初始化TensorFlow持久化类。
# saver = tf.train.Saver()
# config = tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
# sess = tf.Session(config = config)
# with sess:
#     tf.global_variables_initializer().run()
        
#     head = 0
#     for i in range(TRAINING_STEPS):
#         tail = head+BATCH_SIZE
#         if tail > dataSize:
#             xs = np.concatenate((trainX[head: BATCH_SIZE], trainX[0: tail-BATCH_SIZE]))
#             ys = np.concatenate((trainY[head: BATCH_SIZE], trainY[0: tail-BATCH_SIZE]))
#             head = tail - BATCH_SIZE
#         else:
#             xs, ys = trainX[head: head+BATCH_SIZE-1], trainY[head: head+BATCH_SIZE-1]
#             head = tail
            
#         _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
#         if i % 1000 == 0:
#             print("After", step," training step(s), loss on training batch is ", loss_value)
#         if i % 5000 == 0:
#             saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

In [8]:
# config = tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
# restoredSession = tf.Session(config=config)
# restoredSaver = tf.train.import_meta_graph(os.path.join(MODEL_SAVE_PATH, 'tianchi_model-34002.meta'))
# restoredSaver.restore(restoredSession, os.path.join(MODEL_SAVE_PATH, 'tianchi_model-34002'))
# restoredSession.run([loss], feed_dict={x: testX, y_: testY})
# continueTrain(trainX, trainY, restoredSession, 5000)